<a href="https://colab.research.google.com/github/Karan1114Anand/NLP_Project_IEEE/blob/main/Text_Preprocessing_%7C_NLP_%7C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
import pandas as pd
import numpy as np

In [80]:
df = pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Lower Casing in NLP  reduces vocabulary size and improving model efficiency. *Treats words with different cases as the same. It dose not require having to go through the files of different cases.*
This normalization simplifies subsequent processing steps, such as tokenization and feature extraction, leading to more accurate and robust NLP models.

In [82]:
df["review"].str.lower()

,review
0,one of the other reviewers has mentioned that ...
1,a wonderful little production. <br /><br />the...
2,i thought this was a wonderful way to spend ti...
3,basically there's a family where a little boy ...
4,"petter mattei's ""love in the time of money"" is..."
...,...
49995,i thought this movie did a down right good job...
49996,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,i am a catholic taught in parochial elementary...
49998,i'm going to have to disagree with the previou...


Removing HTML tags ensures that, when working with web data...the html tags don't confuse the model resulting in bad traning or even a failed one.


In [83]:
import re

def remove_html(text):
  pattern = re.compile('<.*?>')
  return pattern.sub(r'', text)

In [84]:
text = "<html><body><p> Movie 1</p><p> Actor - Aamir Khan</p><p> Click here to <a href='http://google.com'>download</a></p></body></html>"
text

"<html><body><p> Movie 1</p><p> Actor - Aamir Khan</p><p> Click here to <a href='http://google.com'>download</a></p></body></html>"

In [85]:
remove_html(text)

' Movie 1 Actor - Aamir Khan Click here to download'

We can Also Apply this Function to Whole Corpus.

In [ ]:
df['review']= df['review'].apply(remove_html)

In [7]:
#To remove the url, a similar code is used to define the function
def remove_url(text):
  pattern = re.compile(r'https?://\S+|www\.\S+')
  return pattern.sub(r'', text)

In [8]:
text1 = 'Check out my notebook https://www.kaggle.com/campusx/notebook8223fc1abb'
text2 = 'Check out my notebook http://www.kaggle.com/campusx/notebook8223fc1abb'
text3 = 'Google search here www.google.com'
text4 = 'For notebook click https://www.kaggle.com/campusx/notebook8223fc1abb to search check www.google.com'

In [9]:
print(remove_url(text1))
print(remove_url(text2))
print(remove_url(text3))
print(remove_url(text4))

NameError: name 're' is not defined

Removing punctuation marks is essential in NLP text preprocessing to enhance the accuracy and efficiency of analysis. Punctuation marks like commas, periods, and quotation marks carry little semantic meaning and can introduce noise into the dataset. By removing them, the text becomes cleaner and more uniform, making it easier for machine learning models to extract meaningful features and patterns. Additionally, removing punctuation aids in standardizing the text, ensuring consistency across documents and improving the overall performance of NLP tasks such as sentiment analysis, text classification, and named entity recognition.

In [10]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
punc=string.punctuation

In [12]:
def remove_punc(text):
  return text.translate(str.maketrans('','',punc))

In [13]:
text = "The quick brown fox jumps over the lazy dog. However, the dog doesn't seem impressed! Oh no, it just yawned. How disappointing! Maybe a squirrel would elicit a reaction. Alas, the fox is out of luck."
text

"The quick brown fox jumps over the lazy dog. However, the dog doesn't seem impressed! Oh no, it just yawned. How disappointing! Maybe a squirrel would elicit a reaction. Alas, the fox is out of luck."

In [14]:
alpha = remove_punc(text)
alpha

'The quick brown fox jumps over the lazy dog However the dog doesnt seem impressed Oh no it just yawned How disappointing Maybe a squirrel would elicit a reaction Alas the fox is out of luck'

In [15]:
# can also directly use string.punctuation, reduces the use of a variable making the lines of code a lil less
import string
def remove_pun(text):
  return text.translate(str.maketrans('','',string.punctuation))

In [16]:
beta = remove_pun(text)
beta

'The quick brown fox jumps over the lazy dog However the dog doesnt seem impressed Oh no it just yawned How disappointing Maybe a squirrel would elicit a reaction Alas the fox is out of luck'

Handling chat words or *Slangs* is essential to help model understand the data better if it contains such words and reducing noises. *Important in sentiment analysis*

In [17]:
text = 'IMHO he is the best'
text1 = 'FYI nothing matters tbh'

In [18]:
import re
import requests

# 🔹 Step 1: Load slang dictionary from GitHub once
RAW_URL = "https://raw.githubusercontent.com/rishabhverma17/sms_slang_translator/master/slang.txt"

def _load_slang_dict():
    slang = {}
    text = requests.get(RAW_URL, timeout=20).text
    for line in text.splitlines():
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        if "=" in line:
            k, v = line.split("=", 1)
        elif "->" in line:
            k, v = line.split("->", 1)
        elif "\t" in line:
            k, v = line.split("\t", 1)
        elif " - " in line:
            k, v = line.split(" - ", 1)
        else:
            parts = line.split(None, 1)
            if len(parts) != 2:
                continue
            k, v = parts
        slang[k.strip().upper()] = v.strip()
    return slang

# build dictionary once
CHAT_WORDS = _load_slang_dict()

# 🔹 Step 2: Translator function (no mapping arg needed)
_TOKENIZER = re.compile(r"\w+|[^\w\s]+|\s+")

def chat_conversion(text):
    tokens = _TOKENIZER.findall(text)
    out = []
    for tok in tokens:
        if tok and tok[0].isalnum():
            out.append(CHAT_WORDS.get(tok.upper(), tok))
        else:
            out.append(tok)
    return "".join(out)

# 🔹 Step 3: Example usage
print(chat_conversion(text))
print(chat_conversion(text1))


In My Honest/Humble Opinion he is the best
For Your Information nothing matters To Be Honest


In [19]:
#applying lowercasing and removing punctuation
alp = remove_pun(chat_conversion(text).lower())
alp1 = remove_pun(chat_conversion(text1).lower())
alp,alp1

('in my honesthumble opinion he is the best',
 'for your information nothing matters to be honest')

Spelling correction

In [20]:
# Incorrect text
incorrect_text = 'ceertain conditionas duriing seveal ggenerations aree moodified in the saame maner.'
print(incorrect_text)
# Text 2
incorrect_text2 = 'The cat sat on the cuchion. while plyaiing'

ceertain conditionas duriing seveal ggenerations aree moodified in the saame maner.


In [21]:
from textblob import TextBlob
textblb = TextBlob(incorrect_text)
textblb1 = TextBlob(incorrect_text2)
print(textblb.correct().string)

certain conditions during several generations are modified in the same manner.


In [22]:
print(TextBlob(incorrect_text2).correct().string)

The cat sat on the cushion. while playing


Handling stop words like the, and , is

In [44]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [46]:
from nltk.corpus import stopwords
stopword = stopwords.words('english')
# Function
def remove_stopwords(text):
    new_text = []

    for word in text.split():
        if word in stopword:
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

In [47]:
# Text
text = 'probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it\'s not preachy or boring. it just never gets old, despite my having seen it some 15 or more times'
print(f'Text With Stop Words :{text}')
# Calling Function
remove_stopwords(text)

Text With Stop Words :probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it's not preachy or boring. it just never gets old, despite my having seen it some 15 or more times


'probably  all-time favorite movie,  story  selflessness, sacrifice  dedication   noble cause,    preachy  boring.   never gets old, despite   seen   15   times'

Emojis convey valuable information about sentiment, emotion, and context in text data, especially in informal communication channels like social media. However, they pose challenges for NLP algorithms due to their non-textual nature. Preprocessing involves converting emojis into meaningful representations, such as replacing them with textual descriptions or mapping them to specific sentiment categories.

In [24]:
#Removing emoji
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [26]:
# Texts
text = "Loved the movie. It was 😘"
text1 = 'Python is 🔥'
print(text ,'\n', text1)

# Remove Emojies using Fucntion
print(remove_emoji(text))
remove_emoji(text1)

Loved the movie. It was 😘 
 Python is 🔥
Loved the movie. It was 


'Python is '

In [27]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 6.4 MB/s eta 0:00:00


In [29]:
#converts emoji into their description using the demojize tool
import emoji
print(emoji.demojize(text))
print(emoji.demojize(text1))

Loved the movie. It was :face_blowing_a_kiss:
Python is :fire:


Tokenization is a crucial step in NLP text preprocessing where text is segmented into smaller units, typically words or subwords, known as tokens. This process is essential for several reasons. Firstly, it breaks down the text into manageable units for analysis and processing. Secondly, it standardizes the representation of words, enabling consistency in language modeling tasks. Additionally, tokenization forms the basis for feature extraction and modeling in NLP, facilitating tasks such as sentiment analysis, named entity recognition, and machine translation. Overall, tokenization plays a fundamental role in preparing text data for further analysis and modeling in NLP applications.

We Generally do 2 Type of tokenization 1. Word tokenization 2. Sentence Tokenization


In [41]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [38]:
#1 using NLTK
from nltk.tokenize import word_tokenize,sent_tokenize

In [39]:
sentance = ' I am going to visit Delhi!'
#lowercasing and punctuation removal
sentance = remove_punc(sentance.lower())



In [42]:
#calling the tool
word_tokenize(sentance)

['i', 'am', 'going', 'to', 'visit', 'delhi']

In [48]:
text = """Lorem Ipsum is simply dummy text of the printing and typesetting industry?
Lorem Ipsum has been the industry's standard dummy text ever since the 1500s,
when an unknown printer took a galley of type and scrambled it to make a type specimen book."""

In [49]:
#calling the token
sent_tokenize(text)

['Lorem Ipsum is simply dummy text of the printing and typesetting industry?',
 "Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, \nwhen an unknown printer took a galley of type and scrambled it to make a type specimen book."]

In [55]:
#applying other features aswell
sent_tokenize(remove_punc(remove_stopwords(text.lower())))

['lorem ipsum  simply dummy text   printing  typesetting industry lorem ipsum    industrys standard dummy text ever since  1500s   unknown printer took  galley  type  scrambled   make  type specimen book']

In [58]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [59]:
sent5 = 'I have a Ph.D in A.I'
sent6 = "We're here to help! mail us at nks@gmail.com"
sent7 = 'A 5km ride cost $10.50'

In [60]:
doc1 = nlp(sent5)
doc2 = nlp(sent6)
doc3 = nlp(sent7)

In [61]:
for token in doc2:
    print(token.text)

We
're
here
to
help
!
mail
us
at
nks@gmail.com


Stemming is a text preprocessing technique in NLP used to reduce words to their root or base form, known as a stem, by removing suffixes.

In [63]:
from nltk.stem.porter import PorterStemmer

In [64]:
stemmer = PorterStemmer()

# This Function Will Stem Words
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

In [65]:
st = "walk walks walking walked"
# Calling Function
stem_words(st)

'walk walk walk walk'

In [69]:
text = """probably my alltime favorite movie a story of selflessness sacrifice and dedication to a noble cause but its not preachy
or boring it just never gets old despite my having seen it some 15 or more times in the last 25 years paul lukas performance brings
 tears to my eyes and bette davis in one of her very few truly sympathetic roles is a delight the kids are as grandma says more like
 dressedup midgets than children but that only makes them more fun to watch and the mothers slow awakening to whats happening in the
 world and under her own roof is believable and startling if i had a dozen thumbs theyd all be up for this movie"""
print(text)

# Calling Function
stem_words(remove_punc(remove_stopwords(text.lower())))

probably my alltime favorite movie a story of selflessness sacrifice and dedication to a noble cause but its not preachy 
or boring it just never gets old despite my having seen it some 15 or more times in the last 25 years paul lukas performance brings
 tears to my eyes and bette davis in one of her very few truly sympathetic roles is a delight the kids are as grandma says more like 
 dressedup midgets than children but that only makes them more fun to watch and the mothers slow awakening to whats happening in the 
 world and under her own roof is believable and startling if i had a dozen thumbs theyd all be up for this movie


'probabl alltim favorit movi stori selfless sacrific dedic nobl caus preachi bore never get old despit seen 15 time last 25 year paul luka perform bring tear eye bett davi one truli sympathet role delight kid grandma say like dressedup midget children make fun watch mother slow awaken what happen world roof believ startl dozen thumb theyd movi'

In [73]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [74]:
# We Will Import WordNetLemmatizer from NLTK Library.
from nltk.stem import WordNetLemmatizer
# Intilize Lemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

# Sentence
sentence = "He was running and eating at same time. He has bad habit of swimming after playing long hours in the Sun."

# Intilize Punctuation
punctuations="?:!.,;"

# Tokenize Word
sentence_words = nltk.word_tokenize(sentence)

# Using a Loop to Remove Punctuations.
for word in sentence_words:
    if word in punctuations:
        sentence_words.remove(word)
# Printing Word and Lemmatized Word
print("{0:20}{1:20}".format("Word","Lemma"))
for word in sentence_words:
    print ("{0:20}{1:20}".format(word,wordnet_lemmatizer.lemmatize(word,pos='v')))


Word                Lemma               
He                  He                  
was                 be                  
running             run                 
and                 and                 
eating              eat                 
at                  at                  
same                same                
time                time                
He                  He                  
has                 have                
bad                 bad                 
habit               habit               
of                  of                  
swimming            swim                
after               after               
playing             play                
long                long                
hours               hours               
in                  in                  
the                 the                 
Sun                 Sun                 


Text cleaning.
All the functions like removing html, lowercasing, removing numbers, special character an be done by writing a code for one single function.

In [86]:
import re
import string
from bs4 import BeautifulSoup

def clean_text(text):
  text = text.lower() #lowercasing
  text = re.sub(r'\d+','',text) #removes numbers
  text = text.translate(str.maketrans('','',string.punctuation))#Removes punctuatutions
  text = re.sub(r'\W','',text) #removes special characters
  text = BeautifulSoup(text, "html.parser").get_text() #removes html tags
  return text

In [89]:
df["clean_review"] = df["review"].apply(clean_text)
df["clean_review"]

,clean_review
0,oneoftheotherreviewershasmentionedthatafterwat...
1,awonderfullittleproductionbrbrthefilmingtechni...
2,ithoughtthiswasawonderfulwaytospendtimeonatooh...
3,basicallytheresafamilywherealittleboyjakethink...
4,pettermatteisloveinthetimeofmoneyisavisuallyst...
...,...
49995,ithoughtthismoviedidadownrightgoodjobitwasntas...
49996,badplotbaddialoguebadactingidioticdirectingthe...
49997,iamacatholictaughtinparochialelementaryschools...
49998,imgoingtohavetodisagreewiththepreviouscommenta...


In [92]:
#applying tokenizer tool
df["tokens"] = df["review"].apply(word_tokenize)

In [93]:
df["tokens"]

,tokens
0,"[One, of, the, other, reviewers, has, mentione..."
1,"[A, wonderful, little, production, ., <, br, /..."
2,"[I, thought, this, was, a, wonderful, way, to,..."
3,"[Basically, there, 's, a, family, where, a, li..."
4,"[Petter, Mattei, 's, ``, Love, in, the, Time, ..."
...,...
49995,"[I, thought, this, movie, did, a, down, right,..."
49996,"[Bad, plot, ,, bad, dialogue, ,, bad, acting, ..."
49997,"[I, am, a, Catholic, taught, in, parochial, el..."
49998,"[I, 'm, going, to, have, to, disagree, with, t..."
